In [1]:
#from sympy.parsing.sympy_parser import parse_expr
#import sympy
from sympy.logic.inference import satisfiable
import backprop
import numbs

In [2]:
#A, B = sympy.symbols('A,B')
#expr = sympy.Rational(A, B)
#expr.func
#parse_expr("A/B", evaluate=False).func

stree = backprop.BinaryOperatorSyntaxTree('/', backprop.UnknownSyntaxTree('A'), backprop.UnknownSyntaxTree('B'))
stree_pr  = stree.diff().simplify()
stree_pr2 = stree_pr.diff().simplify()
print(str(stree))
print(str(stree_pr))
print(str(stree_pr2))

(A(x) / B(x))
(((A'(x) * B(x)) - (A(x) * B'(x))) / (B(x) ^ 2))
((((((A''(x) * B(x)) + (A'(x) * B'(x))) - ((A'(x) * B'(x)) + (A(x) * B''(x)))) * (B(x) ^ 2)) - (((A'(x) * B(x)) - (A(x) * B'(x))) * ((2 * (B(x) ^ 1)) * B'(x)))) / (B(x) ^ 4))


In [3]:
symbmapper = backprop.SymbolMapper()

peak_x   = 0.20827333333333353
root_pr  = 0.22
#root_pr  = 0.00781025
#root_pr2 = 0.0135277

roots = [-numbs.INFTY, -root_pr, -peak_x, 0, peak_x, root_pr, numbs.INFTY]

"""pk_boolexpr  = stree.backprop_sign(symbmapper, -numbs.INFTY, -root_pr, '+')
pk_boolexpr &= stree.backprop_sign(symbmapper, -root_pr, -peak_x, '+')
pk_boolexpr &= stree.backprop_sign(symbmapper, -peak_x, 0, '+')
pk_boolexpr &= stree.backprop_sign(symbmapper, 0, peak_x, '-')
pk_boolexpr &= stree.backprop_sign(symbmapper, peak_x, root_pr, '-')
pk_boolexpr &= stree.backprop_sign(symbmapper, root_pr, numbs.INFTY, '-')

pk_boolexpr &= stree_pr.backprop_sign(symbmapper, -numbs.INFTY, -root_pr, '+')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, -root_pr, -peak_x, '+')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, -peak_x, 0, '-')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, 0, peak_x, '-')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, peak_x, root_pr, '+')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, root_pr, numbs.INFTY, '+')

pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, -numbs.INFTY, -root_pr, '+')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, -root_pr, -peak_x, '-')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, -peak_x, 0, '-')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, 0, peak_x, '+')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, peak_x, root_pr, '+')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, root_pr, numbs.INFTY, '-')"""

pk_boolexpr  = stree.backprop_sign(symbmapper, -numbs.INFTY, 0, '+')
pk_boolexpr &= stree.backprop_sign(symbmapper, 0, numbs.INFTY, '-')

"""
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, -numbs.INFTY, -peak_x, '+')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, -peak_x, 0, '-')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, 0, peak_x, '-')
pk_boolexpr &= stree_pr.backprop_sign(symbmapper, peak_x, numbs.INFTY, '+')

pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, -numbs.INFTY, -root_pr, '+')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, -root_pr, 0, '-')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, 0, root_pr, '+')
pk_boolexpr &= stree_pr2.backprop_sign(symbmapper, root_pr, numbs.INFTY, '-')
"""
pk_boolexpr

((X0 & X2) | (X1 & X3)) & ((X4 & X7) | (X5 & X6))

In [4]:
for X_symb_a in pk_boolexpr.free_symbols:
    for X_symb_b in pk_boolexpr.free_symbols:
        if X_symb_a == X_symb_b: continue
        if symbmapper.symb_to_prop[X_symb_a].fights( symbmapper.symb_to_prop[X_symb_b] ):
            print(f"{symbmapper.symb_to_prop[X_symb_a]} - {symbmapper.symb_to_prop[X_symb_b]}")
            pk_boolexpr &= ~( X_symb_a & X_symb_b )

B(x)>0 [0,INFTY] - B(x)<0 [0,INFTY]
A(x)>0 [0,INFTY] - A(x)<0 [0,INFTY]
B(x)<0 [-INFTY,0] - B(x)>0 [-INFTY,0]
B(x)<0 [0,INFTY] - B(x)>0 [0,INFTY]
A(x)<0 [-INFTY,0] - A(x)>0 [-INFTY,0]
B(x)>0 [-INFTY,0] - B(x)<0 [-INFTY,0]
A(x)>0 [-INFTY,0] - A(x)<0 [-INFTY,0]
A(x)<0 [0,INFTY] - A(x)>0 [0,INFTY]


In [5]:
unknowns_labels = ["A", "B"]#, "A'", "B'", "A''", "B''"] 

"""
for i in range(len(roots)-1):
    for l in unknowns_labels:
        c_gr = backprop.PropositionalConstraint(
            backprop.UnknownSyntaxTree(l),
            backprop.ConstantSyntaxTree(0),
            '>', roots[i], roots[i+1]
        )
        c_lw = backprop.PropositionalConstraint(
            backprop.UnknownSyntaxTree(l),
            backprop.ConstantSyntaxTree(0),
            '<', roots[i], roots[i+1]
        )
        pk_boolexpr &= ( symbmapper.map_as_symbol(c_gr) | symbmapper.map_as_symbol(c_lw) )"""
pk_boolexpr

~(X0 & X1) & ~(X2 & X3) & ~(X4 & X5) & ~(X6 & X7) & ((X0 & X2) | (X1 & X3)) & ((X4 & X7) | (X5 & X6))

In [6]:
allmodels = satisfiable(pk_boolexpr, algorithm='minisat22', minimal=True, all_models=True)

n = 0
while n < 10:
    
    try: models = next(allmodels)
    except RuntimeError:
        print('No more models.')
        break

    if models:
        constrs = set()
        constrs_map = {}
        
        for symb in models.keys():
            if models[symb]:
                constr = symbmapper.symb_to_prop[symb]
                constrs.add(constr)
                constr_f_str = str(constr.f)
                if constr_f_str not in constrs_map.keys(): constrs_map[constr_f_str] = []
                constrs_map[constr_f_str].append(constr)
        
        """some_dom_constr = True
        while some_dom_constr:

            dom_constrs = []
            for ci in constrs:
                for cj in constrs:
                    if ci == cj: continue
                    if ci.is_dominated_by(cj):
                        dom_constrs.append(ci)
                        break
            
            for dc in dom_constrs:
                constrs.remove(dc)
            
            some_dom_constr = len(dom_constrs) > 0"""
        
        #for c in constrs:
        #    print(c)
        print(f"--- Model {n} ---")
        for k in constrs_map.keys():
            for c in constrs_map[k]: print(c)
            print()
        print()
                
    else:
        print('Unsat')
        break

    n += 1

--- Model 0 ---
B(x)>0 [0,INFTY]
B(x)<0 [-INFTY,0]

A(x)<0 [-INFTY,0]
A(x)<0 [0,INFTY]


--- Model 1 ---
B(x)>0 [0,INFTY]
B(x)>0 [-INFTY,0]

A(x)<0 [0,INFTY]
A(x)>0 [-INFTY,0]


--- Model 2 ---
B(x)<0 [0,INFTY]
B(x)>0 [-INFTY,0]

A(x)>0 [-INFTY,0]
A(x)>0 [0,INFTY]


--- Model 3 ---
B(x)<0 [0,INFTY]
B(x)<0 [-INFTY,0]

A(x)<0 [-INFTY,0]
A(x)>0 [0,INFTY]


No more models.
